In [1]:
import numpy as np
import os
import matplotlib.pyplot as plt
from tensorflow import keras
import tensorflow as tf

In [2]:
data = np.load('/gpfs/data1/lianggp/lir/solar_data/code/SolarSat/solarsat_point_train.npz')
x_train = data['arr_0']
y_train = data['arr_1']
data = np.load('/gpfs/data1/lianggp/lir/solar_data/code/SolarSat/solarsat_point_test.npz')
x_test = data['arr_0']
y_test = data['arr_1']

In [3]:
print(x_train.shape, y_train.shape, x_test.shape, y_test.shape)

(24715, 1, 1, 8) (24715, 1, 1, 12) (8608, 1, 5, 8) (8608, 1, 1, 12)


In [4]:
x_test=np.transpose(x_test[:,0,:,:], (0,2,1))
y_test=np.transpose(y_test[:,0,:,:], (0,2,1))
print(x_train.shape, y_train.shape, x_test.shape, y_test.shape)

(24715, 1, 1, 8) (24715, 1, 1, 12) (8608, 8, 5) (8608, 12, 1)


In [5]:
from tensorflow.keras import layers
def build_lstm_model():
    # Input layer: Expect input shape (8, 5), where 8 is the number of time steps and 5 is the number of features per time step.
    inputs = keras.Input(shape=(8, 5))

    # LSTM layer: Let's assume we use 20 units and want to keep the sequence
    x = layers.LSTM(20, return_sequences=True)(inputs)

    # Additional LSTM layer: Adjust to return a single feature, still keeping all time steps
    x = layers.LSTM(1, return_sequences=True)(x)

    # Output shaping: Assume we need to adapt time steps via a Dense layer
    # Here, we should decide if we really need to expand to 12 steps, which usually would require a different approach since LSTM doesn't change time steps count
    # Assume this was a misunderstanding and we focus on adjusting features only.
    outputs = layers.TimeDistributed(layers.Dense(12))(x)  # This operation will not change the number of time steps but will project each time step's output to a 12-dimensional vector.

    model = keras.Model(inputs=inputs, outputs=outputs)
    model.compile(optimizer='adam', loss='mse')  # Compile with an optimizer and loss function

    return model

In [6]:
model = build_lstm_model()
model.summary()

2024-05-18 13:59:13.897120: I tensorflow/stream_executor/platform/default/dso_loader.cc:44] Successfully opened dynamic library libcuda.so.1
2024-05-18 13:59:13.910132: I tensorflow/core/common_runtime/gpu/gpu_device.cc:1555] Found device 0 with properties: 
pciBusID: 0000:2f:00.0 name: Tesla V100-PCIE-16GB computeCapability: 7.0
coreClock: 1.38GHz coreCount: 80 deviceMemorySize: 15.78GiB deviceMemoryBandwidth: 836.37GiB/s
2024-05-18 13:59:13.910798: I tensorflow/core/common_runtime/gpu/gpu_device.cc:1555] Found device 1 with properties: 
pciBusID: 0000:86:00.0 name: Tesla V100-PCIE-16GB computeCapability: 7.0
coreClock: 1.38GHz coreCount: 80 deviceMemorySize: 15.78GiB deviceMemoryBandwidth: 836.37GiB/s
2024-05-18 13:59:13.934384: I tensorflow/stream_executor/platform/default/dso_loader.cc:44] Successfully opened dynamic library libcudart.so.10.1
2024-05-18 13:59:14.263635: I tensorflow/stream_executor/platform/default/dso_loader.cc:44] Successfully opened dynamic library libcublas.so.

: 

: 

: 

In [ ]:
def build_model_bigru(trainX):
    model = Sequential()
    model.add(Bidirectional(GRU(32,  input_shape=(trainX.shape[1], trainX.shape[2]), return_sequences=True)))
    model.add(Dense(units=25, activation='relu'))
    model.add(Dense(units=5, activation='relu'))
    model.add(Dense(units=1))
    adam = keras.optimizers.Adam(lr=0.001)
    model.compile(optimizer=adam, loss='mse',metrics=[RMSE,BIAS])
    return model